In [1]:
# medir tiempos
%load_ext autotime

time: 1.29 ms (started: 2021-06-14 12:04:08 -05:00)


In [2]:
import findspark
findspark.init("/usr/local/spark/spark-3.1.1-bin-hadoop2.7")    #para linux
#findspark.init()                                                 #para windows

time: 18.9 ms (started: 2021-06-14 12:04:08 -05:00)


In [3]:
from pyspark import SparkConf, SparkContext
# Variable de configuración
conf = SparkConf().setMaster("local[*]").setAppName("GBTRegressor")  \
    .set("spark.driver.maxResultSize","0") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "16g") \
    .set("spark.cores.max", "4")

# iniciamos un contexto spark (solo se ejecuta uno. Para ejecutar otra vez , reiniciar el kernel)
sc = SparkContext(conf = conf)
sc

<SparkContext master=local[*] appName=GBTRegressor>

time: 4.8 s (started: 2021-06-14 12:04:09 -05:00)


In [4]:
from pyspark.sql.types import StringType
from pyspark import SQLContext
# le pasamos el contexto anterior
sqlContext = SQLContext(sc)
import os
path="file:"+os.getcwd()+"/Output"
dfspark = sqlContext.read.format('csv').option("header","true").option("inferSchema","true").load(path+'/*.csv')

time: 20.6 s (started: 2021-06-14 12:04:14 -05:00)


In [5]:
sc.getConf().getAll()

[('spark.executor.memory', '4g'),
 ('spark.app.startTime', '1623690252042'),
 ('spark.executor.id', 'driver'),
 ('spark.driver.memory', '16g'),
 ('spark.cores.max', '4'),
 ('spark.driver.port', '42015'),
 ('spark.rdd.compress', 'True'),
 ('spark.app.id', 'local-1623690253396'),
 ('spark.driver.maxResultSize', '0'),
 ('spark.serializer.objectStreamReset', '100'),
 ('spark.master', 'local[*]'),
 ('spark.submit.pyFiles', ''),
 ('spark.app.name', 'GBTRegressor'),
 ('spark.submit.deployMode', 'client'),
 ('spark.sql.warehouse.dir',
  'file:/home/renzoqamao/my_project_dir/my_project_env/2021-I/InteligenciaArtificial/PracticasCalificadas/PracticaCalificada2-CC421/IA-PracticaCalificada2/spark-warehouse'),
 ('spark.ui.showConsoleProgress', 'true'),
 ('spark.driver.host', '192.168.1.5')]

time: 66.6 ms (started: 2021-06-14 12:04:34 -05:00)


In [6]:
dfspark.printSchema()

root
 |-- fare_amount: double (nullable = true)
 |-- pickup_datetime: string (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- dif_latitude: double (nullable = true)
 |-- dif_longitude: double (nullable = true)
 |-- distancia: double (nullable = true)
 |-- dia_semana: integer (nullable = true)
 |-- hora: integer (nullable = true)
 |-- mes: integer (nullable = true)
 |-- anio: integer (nullable = true)

time: 63.8 ms (started: 2021-06-14 12:04:34 -05:00)


##### creando un vector de caracteristicas

In [7]:
from pyspark.ml.feature import VectorAssembler
ignore=['fare_amount',
        'pickup_datetime',
        #'pickup_longitude',
        #'pickup_latitude',
        #'dropoff_longitude',
        #'dropoff_latitude',
       'dif_latitude',
       'dif_longitude']
vectorAssembler = VectorAssembler(inputCols=[x for x in dfspark.columns  
                  if x not in ignore], outputCol = 'features')
df = vectorAssembler.transform(dfspark)
dfmodel = df.select(['features', 'fare_amount'])

time: 1.41 s (started: 2021-06-14 12:04:34 -05:00)


In [8]:
# Separar la data en 80% entrenamiento y 20% test
dftrain, dftest = dfmodel.randomSplit([0.7, 0.3], seed=0)
print("tenemos %d datos de entrenamiento y %d datos de prueba." % (dftrain.count(), dftest.count()))

tenemos 1551865 datos de entrenamiento y 666459 datos de prueba.
time: 25 s (started: 2021-06-14 12:04:36 -05:00)


### GBTRegressor

In [9]:
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator

gbt = GBTRegressor(labelCol="fare_amount", featuresCol="features", seed=0, lossType="squared")

time: 342 ms (started: 2021-06-14 12:05:01 -05:00)


In [10]:
gbtrparamGrid = (ParamGridBuilder()
             .addGrid(gbt.maxDepth, [12])
             .addGrid(gbt.maxBins, [24])
             .addGrid(gbt.maxIter, [10])
             .build())

time: 1.18 ms (started: 2021-06-14 12:05:01 -05:00)


In [11]:
gbevaluator = RegressionEvaluator(predictionCol="prediction", labelCol="fare_amount", metricName="rmse")

time: 240 ms (started: 2021-06-14 12:05:01 -05:00)


In [12]:
# Create 3-fold CrossValidator
gbcv = CrossValidator(estimator = gbt,
                      estimatorParamMaps = gbtrparamGrid,
                      evaluator = gbevaluator,
                      numFolds = 3)

time: 76.6 ms (started: 2021-06-14 12:05:01 -05:00)


In [13]:
gbcvModel = gbcv.fit(dftrain)

time: 7min 58s (started: 2021-06-14 12:05:01 -05:00)


In [14]:
print(gbcvModel)

CrossValidatorModel_aac00dbde023
time: 670 µs (started: 2021-06-14 12:13:05 -05:00)


In [15]:
gbpredictions = gbcvModel.transform(dftest)

time: 332 ms (started: 2021-06-14 12:13:07 -05:00)


In [16]:
print('RMSE:', gbevaluator.evaluate(gbpredictions))

RMSE: 4.38793600483846
time: 12.3 s (started: 2021-06-14 12:13:08 -05:00)


In [17]:
path="file:"+os.getcwd()+"/GBTRRegressor-Muestra-RMSE-4.3789/"
gbt.save(path+"gbtregressor/")
gbcv.save(path+"gbcrosvalidator/")
gbcvModel.save(path+"Model/")
gbevaluator.save(path+"evaluator/")

time: 3.84 s (started: 2021-06-14 12:20:00 -05:00)


In [18]:
sc.stop()

time: 146 ms (started: 2021-06-14 12:48:05 -05:00)
